<a href="https://colab.research.google.com/github/cfoli/Multi-label-Medical-Image-Classification/blob/main/ChestVision_ViTs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Vision-Transformer-based solutions for assistive medical diagnosis

This project leverages pre-trained vision transformers to build end-to-end solutions for multi-label medical image (i.e., chest x-ray) classification.
***
Approach: load pre-trained vision transformers for image feature extraction and attach classification heads for multi-label prediction.

### Import libraries

---



In [1]:
!pip install lightning torchmetrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 853.6/853.6 kB 74.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 857.3/857.3 kB 62.2 MB/s eta 0:00:00


In [2]:
import torch
import torch.nn.functional as F
import torchvision
from torchvision import transforms, models, datasets
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
from torch.utils.data import random_split
import pytorch_lightning as torch_light
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
import torchmetrics
from torchmetrics import Metric
import os
import shutil
import subprocess
import pandas as pd
from PIL import Image
from transformers import AutoModel


### Define helper functions and containers

---



In [4]:
BASE_DIR = os.path.join(os.getcwd(), "datasets")
# Set float32 matrix multiplication precision as recommended for Tensor Cores
# torch.set_float32_matmul_precision('medium')

configs = {
    "IMAGE_SIZE":   (224, 224),    # Resize images to (W, H)
    "NUM_CHANNELS": 3,             # RGB images
    "NUM_CLASSES":  15,            # Number of output labels

    "BATCH_SIZE": 64,
    "NUM_WORKERS": 10,

    # ImageNet dataset normalization values (for pretrained backbones)
    "MEAN": (0.485, 0.456, 0.406),
    "STD":  (0.229, 0.224, 0.225),

    "LABELS_CSV_DIR": os.path.join(BASE_DIR, "new_labels.csv"),
    "IMG_DIR": os.path.join(BASE_DIR, "resized_images", "resized_images"),

    "TRAIN_PCT": 0.8, # Fraction of data for training
    "VAL_PCT":   0.1, # Fraction of data for validation
    }

# labels = ["Atelectasis", "Cardiomegaly", "Consolidation", "Edema", "Effusion", "Emphysema", "Fibrosis", "Hernia", "Infiltration", "Mass", "No finding", "Nodule", "Pleural_Thickening", "Pneumonia","Pneumothorax"]

MODEL_REGISTRY = {
    "CheXFormer-base": "m42-health/CXformer-base",
    "CheXFormer-small": "m42-health/CXformer-small",
    "ViT-base-16": "google/vit-base-patch16-224",
    "ViT-large-16": "google/vit-large-patch16-224",
    # "DINOv3-ViT16": "facebook/dinov3-vitl16-pretrain-lvd1689m"
}


In [ ]:
model = torchvision.models.efficientnet_b3(weights="DEFAULT")
# print(model) # efficientnet_v2_s, AlexNet
last_module_name, last_module = list(model.named_children())[-1]
print(last_module)


Downloading: "https://download.pytorch.org/models/efficientnet_b3_rwightman-b3899882.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b3_rwightman-b3899882.pth


100%|██████████| 47.2M/47.2M [00:00<00:00, 224MB/s]


Sequential(
  (0): Dropout(p=0.3, inplace=True)
  (1): Linear(in_features=1536, out_features=1000, bias=True)
)


In [ ]:
from transformers import AutoModel, AutoImageProcessor
from PIL import Image

model_name = "google/vit-base-patch16-224"
# Options (Multi-label image classification):
# m42-health/CXformer-base,
# m42-health/CXformer-small
# google/vit-base-patch16-224
# facebook/dinov3-vitl16-pretrain-lvd1689m
# facebook/dinov3-vitb16-pretrain-lvd1689m

image_processor = AutoImageProcessor.from_pretrained(model_name,trust_remote_code=True)
model = AutoModel.from_pretrained(model_name)

# Options (Image-to-text)
# nlpconnect/vit-gpt2-image-captioning


preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

Fast image processor class <class 'transformers.models.vit.image_processing_vit_fast.ViTImageProcessorFast'> is available for this model. Using slow image processor class. To use the fast image processor class set `use_fast=True`.


Loading weights:   0%|          | 0/198 [00:00<?, ?it/s]

ViTModel LOAD REPORT from: google/vit-base-patch16-224
Key                 | Status     | 
--------------------+------------+-
classifier.weight   | UNEXPECTED | 
classifier.bias     | UNEXPECTED | 
pooler.dense.bias   | MISSING    | 
pooler.dense.weight | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.


In [ ]:
print(model)
# last_module_name, last_module = list(model.named_children())[-1]
# model.config.hidden_size

ViTModel(
  (embeddings): ViTEmbeddings(
    (patch_embeddings): ViTPatchEmbeddings(
      (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    )
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (encoder): ViTEncoder(
    (layer): ModuleList(
      (0-11): 12 x ViTLayer(
        (attention): ViTAttention(
          (attention): ViTSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
          )
          (output): ViTSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
          )
        )
        (intermediate): ViTIntermediate(
          (dense): Linear(in_features=768, out_features=3072, bias=True)
          (intermediate_act_fn): GELUActivation()
        )
        (output): ViTOutput(
          (d

In [ ]:

model.eval()

image = Image.open('sample_cxr.png')

image = image_processor(image, return_tensors='pt')
print(image['pixel_values'].shape) # [1,3,518,518]

print("Doing forwardpass...")
output = model(**image).last_hidden_state  # [1, 1374, 768]


In [5]:
class get_pretrained_model(nn.Module):
    def __init__(
        self,
        model_name: str,
        num_classes: int,
        num_layers_to_unfreeze: int = 0):
        super().__init__()

        print(f"Loading pretrained [{model_name}] model")

        self.backbone = AutoModel.from_pretrained(
            MODEL_REGISTRY[model_name],
            trust_remote_code=True)

        hidden_size = self.backbone.config.hidden_size

        # Freeze entire backbone first
        for param in self.backbone.parameters():
            param.requires_grad = False

        # Selectively unfreeze last N layers
        if num_layers_to_unfreeze > 0:
            self._unfreeze_last_n_layers(num_layers_to_unfreeze)

        # Single classification head
        self.classifier = nn.Sequential(
            nn.LayerNorm(hidden_size),
            nn.Dropout(0.4),
            nn.Linear(hidden_size, num_classes) )

    def forward(self, x):
        outputs = self.backbone(x)

        # Use CLS token
        img_embeddings = outputs.last_hidden_state[:, 0]

        logits = self.classifier(img_embeddings)
        return logits

    def _unfreeze_last_n_layers(self, n: int):
        if hasattr(self.backbone, "encoder"):
            encoder_layers = self.backbone.encoder.layer
        elif hasattr(self.backbone, "vision_model"):
            encoder_layers = self.backbone.vision_model.encoder.layer
        else:
            raise ValueError("Cannot find encoder layers in backbone.")

        total_layers = len(encoder_layers)
        n = min(n, total_layers)

        print(f"Unfreezing last {n} of {total_layers} transformer layers.")

        for layer in encoder_layers[-n:]:
            for param in layer.parameters():
                param.requires_grad = True


### Load kaggle credentials (i.e., API) file

---



In [6]:
# Import kaggle API token file
from google.colab import files
files.upload() # This will prompt you to select and upload the file

# Then move and secure the file
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


Saving kaggle.json to kaggle.json


### Create model (i.e. classifier) module

---



In [7]:
class modelModule(torch_light.LightningModule):
    def __init__(self, num_classes, backbone_model_name, num_layers_to_unfreeze):
        super().__init__()
        self.num_classes = num_classes
        self.backbone_model_name = backbone_model_name
        self.num_layers_to_unfreeze = num_layers_to_unfreeze

        # Load a pretrained backbone and replace its final layer
        self.model = get_pretrained_model(
            num_classes = self.num_classes,
            model_name  = self.backbone_model_name,
            num_layers_to_unfreeze = self.num_layers_to_unfreeze)

        # Binary classification loss operating on raw logits
        self.loss_function      = torch.nn.BCEWithLogitsLoss()
        # self.accuracy_function = torchmetrics.Accuracy(task="multilabel", num_labels=self.num_classes)
        # self.f1_score_function = torchmetrics.F1Score(task="multilabel", num_labels=self.num_classes)
        self.accuracy_function  = torchmetrics.classification.MultilabelAccuracy(num_labels=self.num_classes, average="weighted", threshold=0.5)
        self.f1_score_function  = torchmetrics.classification.MultilabelF1Score(num_labels=self.num_classes, average="weighted", threshold=0.5)
        self.auroc_function     = torchmetrics.classification.MultilabelAUROC(num_labels=self.num_classes, average="weighted", thresholds=10)
        self.map_score_function = torchmetrics.classification.MultilabelAveragePrecision(num_labels=self.num_classes, average="weighted", thresholds=10)
        # average options: macro (simple average), micro (sum), weighted (weight by class size, then avg)
        # threshold: Threshold for transforming probability to binary (0,1) predictions. For some metrics (e.g., AUROC), represents the number of thresholds (evenly spaced b/n 0–1) the metric should be computed at (resulting array of values are the averaged to obtain the final score)

    def forward(self, x):
        # Forward pass through the backbone model
        return self.model(x)

    def _common_step(self, batch, batch_idx):
        """
        Shared logic for train / val / test steps.
        Computes loss and evaluation metrics.
        """
        x, y = batch

        # Compute model predictions ()
        y_logits = self.forward(x)
        y_prob    = torch.sigmoid(y_logits)

        # Compute metrics (expects logits + labels)
        loss     = self.loss_function(y_logits, y.float())

        # Compute mean loss over all classes
        # loss     = torchmetrics.aggregation.MeanMetric(self.loss_function(y_hat, y.float()), weight=X.shape[0])
        accuracy = self.accuracy_function(y_prob, y)
        f1_score = self.f1_score_function(y_prob, y)
        auroc    = self.auroc_function(y_prob, y)
        mAP      = self.map_score_function(y_prob, y) # mean average precision

        return loss, y_logits, y, accuracy, f1_score, auroc, mAP

    def training_step(self, batch, batch_idx):
        # Run shared step
        loss, y_logits, y, accuracy, f1_score, auroc, mAP = self._common_step(batch, batch_idx)

        # Log epoch-level training metrics
        self.log_dict(
            {"train_loss": loss, "train_accuracy": accuracy, "train_f1_score": f1_score, "train_auroc": auroc, "train_mAP": mAP},
            on_step=False, on_epoch=True, prog_bar=True)

        # Lightning expects the loss key for backprop
        return {"loss": loss}

    def validation_step(self, batch, batch_idx):
        # Run shared step
        loss, y_logits, y, accuracy, f1_score, auroc, mAP = self._common_step(batch, batch_idx)

        # Log validation metrics
        self.log_dict(
            {"val_loss": loss, "val_accuracy": accuracy,"val_f1_score": f1_score, "val_auroc": auroc, "val_mAP": mAP},
            on_step=False, on_epoch=True, prog_bar=True)

    def test_step(self, batch, batch_idx):
        # Run shared step
        loss, y_logits, y, accuracy, f1_score, auroc, mAP = self._common_step(batch, batch_idx)

        # Log test metrics
        self.log_dict(
            {"test_loss": loss, "test_accuracy": accuracy,"test_f1_score": f1_score, "test_auroc": auroc, "test_mAP": mAP},
            on_step=False, on_epoch=True, prog_bar=True)

    def predict_step(self, batch, batch_idx):
        """
        Prediction logic used by trainer.predict().
        Returns model outputs without computing loss.
        """
        x = batch if not isinstance(batch, (tuple, list)) else batch[0]
        logits = self.forward(x)

        # Convert logits to probabilities for inference
        probs = torch.sigmoid(logits)

        return probs

    def configure_optimizers(self):
        # Optimizer over all trainable parameters
        optimizer = optim.Adam(self.parameters(), lr=3e-5)
        return optimizer


### Create data module

---



In [8]:

class createMultiLabelTorchDataset(Dataset):
    def __init__(self, labels_csv_dir, img_dir, transform=None):
        self.df = pd.read_csv(labels_csv_dir)
        self.label_cols = self.df.columns.drop("Path") # Get names of labels (i.e., cols, excluding img name)
        self.img_dir     = img_dir
        self.transform   = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image_id = row["Path"]

        img_path = os.path.join(self.img_dir, image_id)
        image = Image.open(img_path).convert("RGB")  # Forces RGB (3 channels)

        if self.transform:
            image = self.transform(image)

        # multi-hot target
        target = row[self.label_cols].values.astype(int) # extract multihot target array from csv file and convert to int
        target = torch.tensor(target, dtype=torch.long) # convert multihot target array to long tensor

        return image, target

class dataModule(torch_light.LightningDataModule):
  def __init__(self, batch_size, num_classes):
    super().__init__()

    self.num_classes = num_classes
    # self.validation_size = validation_size # validation_size = 0.1
    self.batch_size = batch_size

    self.train_transforms = transforms.Compose([
                transforms.RandomAffine(degrees=40, translate=(0.01, 0.12), shear=0.05),
                transforms.RandomHorizontalFlip(),
                transforms.RandomVerticalFlip(),
                transforms.Normalize(configs["MEAN"], configs["STD"], inplace=True),
                transforms.ToTensor(),
                transforms.RandomErasing(inplace=True)])

    self.valid_transforms = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(configs["MEAN"], configs["STD"], inplace=True),
                ])
    self.test_transforms = self.valid_transforms

  def prepare_data(self):
      # Download datasets from Kaggle if not already downloaded
      if os.path.exists(BASE_DIR):
          return  # already downloaded

      # Kaggle expects the API key (i.e., json) to live in a directory like ~/.kaggle/kaggle.json. So we create that directory and copy the loaded Kaggle file there
      kaggle_dir  = os.path.join(os.path.expanduser("~"), ".kaggle")
      kaggle_json = os.path.join(kaggle_dir, "kaggle.json")

      if not os.path.exists(kaggle_json):
          os.makedirs(kaggle_dir, exist_ok=True)
          shutil.copyfile("kaggle.json", kaggle_json)
          os.chmod(kaggle_json, 0o600)

      print("Downloading dataset from Kaggle...")

      # Download datasets from Kaggle
      subprocess.run(
          ["kaggle", "datasets", "download","-q", "rahulogoel/nih-balanced-and-resized-chest-x-rays", "-p", BASE_DIR, "--unzip"],
          check=True)
      print("Download complete!")

  def setup(self, stage=None):

    full_dataset = createMultiLabelTorchDataset(
        labels_csv_dir = configs["LABELS_CSV_DIR"],
        img_dir = configs["IMG_DIR"],
        transform = self.train_transforms # temporarily assign train transforms
        )

    n_total = len(full_dataset)
    train_set_size = int(configs["TRAIN_PCT"] * n_total)
    val_set_size   = int(configs["VAL_PCT"] * n_total)
    test_set_size  = n_total - train_set_size - val_set_size

    generator = torch.Generator().manual_seed(42)

    train_subset, val_subset, test_subset = random_split(
        full_dataset,[train_set_size, val_set_size, test_set_size],
        generator=generator)

    # Assign transforms AFTER split
    train_subset.dataset.transform = self.train_transforms
    val_subset.dataset.transform   = self.valid_transforms
    test_subset.dataset.transform  = self.test_transforms

    if stage in (None, "fit"):
        self.train_dataset = train_subset
        self.val_dataset   = val_subset

    if stage in (None, "test"):
        self.test_dataset  = test_subset


  def train_dataloader(self):
    # create training data loader object from training dataset
    return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True, num_workers = configs["NUM_WORKERS"])

  def val_dataloader(self):
    return DataLoader(self.val_dataset, batch_size=self.batch_size, shuffle=False, num_workers = configs["NUM_WORKERS"])

  def test_dataloader(self):
    return DataLoader(self.test_dataset, batch_size=self.batch_size, shuffle=False, num_workers = configs["NUM_WORKERS"])

### Train model

---



In [ ]:

model_ = modelModule(
    num_classes = configs["NUM_CLASSES"],
    backbone_model_name = "DINOv3-ViT16", # ViT-large-16, CheXFormer-base, CheXFormer-small
    num_layers_to_unfreeze = 3)

data_module = dataModule(
    batch_size      = configs["BATCH_SIZE"],
    num_classes     = configs["NUM_CLASSES"])

trainer_ = torch_light.Trainer(
    accelerator = "gpu",
    devices     = "auto",
    strategy    = "auto",
    max_epochs  = 10,
    callbacks   = [EarlyStopping(patience=3, verbose=False, monitor="val_loss")],
    precision   = "16")

trainer_.fit(model_, data_module)
trainer_.validate(model_, data_module)
trainer_.test(model_, data_module)


Loading pretrained [DINOv3-ViT16] model


OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/facebook/dinov3-vitl16-pretrain-lvd1689m.
401 Client Error. (Request ID: Root=1-6991d9c9-10d537914e93d507370f6d9a;c43d88be-7ac7-4a2b-9bac-a032d1c07b5c)

Cannot access gated repo for url https://huggingface.co/facebook/dinov3-vitl16-pretrain-lvd1689m/resolve/main/config.json.
Access to model facebook/dinov3-vitl16-pretrain-lvd1689m is restricted. You must have access to it and be authenticated to access it. Please log in.

In [ ]:
import torch
print(torch.cuda.is_available())
torch.cuda.empty_cache()


True


### Train and compare different models

In [9]:

SAVE_ROOT = '/content/drive/MyDrive/ML Projects/Multi-label Diagnosis (ChestXRay)/Trained models'

os.makedirs(SAVE_ROOT, exist_ok=True)

# Selection criteria: top1 acc>85%; GFLOPS<500:
backbones = ["CheXFormer-base", "CheXFormer-small", "ViT-base-16",  "ViT-large-16"]

results = []

for backbone in backbones:
    print(f"\n================ {backbone} =================\n")

    save_dir = os.path.join(SAVE_ROOT, "Vision Transformers")
    os.makedirs(save_dir, exist_ok=True)

    # ------------------
    # Model
    # ------------------
    model_ = modelModule(
        num_classes            = configs["NUM_CLASSES"],
        backbone_model_name    = backbone,
        num_layers_to_unfreeze = 2)

    data_module = dataModule(
        batch_size  = configs["BATCH_SIZE"],
        num_classes = configs["NUM_CLASSES"])

    # ------------------
    # Callbacks
    # ------------------
    checkpoint_cb  = ModelCheckpoint(
        dirpath    = save_dir,
        filename   = f"{backbone}",
        monitor    = "val_loss",        # 🔥 use mAP as primary metric
        mode       = "max",
        save_top_k = 1,
        save_last  = False)

    early_stop_cb = EarlyStopping(
        monitor   = "val_loss",
        mode      = "max",
        patience  = 3)

    # ------------------
    # Trainer
    # ------------------
    trainer_ = torch_light.Trainer(
        accelerator         = "auto",
        devices             = "auto",
        strategy            = "auto",
        max_epochs          = 10,
        precision           = "16",
        callbacks           = [checkpoint_cb, early_stop_cb],
        enable_progress_bar = True,
        logger              = False)

    # ------------------
    # Train
    # ------------------
    trainer_.fit(model_, data_module)

    # ------------------
    # Load best model
    # ------------------
    best_ckpt_path = checkpoint_cb.best_model_path
    print(f"Best checkpoint: {best_ckpt_path}")

    best_model = modelModule.load_from_checkpoint(
        best_ckpt_path,
        num_classes         = configs["NUM_CLASSES"],
        backbone_model_name = backbone,
        num_layers_to_unfreeze = 2)

    # ------------------
    # Validate & Test
    # ------------------
    val_metrics  = trainer_.validate(best_model, data_module, verbose=False)[0]
    test_metrics = trainer_.test(best_model, data_module, verbose=False)[0]

    # ------------------
    # Store results
    # ------------------
    results.append({
        "model": backbone,
        "checkpoint": best_ckpt_path,

        "val_loss":   val_metrics.get("val_loss"),
        "val_acc":    val_metrics.get("val_acc"),
        "val_f1":     val_metrics.get("val_f1"),
        "val_mAP":    val_metrics.get("val_mAP"),
        "val_auroc":  val_metrics.get("val_auroc"),

        "test_loss":  test_metrics.get("test_loss"),
        "test_acc":   test_metrics.get("test_acc"),
        "test_f1":    test_metrics.get("test_f1"),
        "test_map":   test_metrics.get("test_mAP"),
        "test_auroc": test_metrics.get("test_auroc")})



================ CheXFormer-base =================

Loading pretrained [CheXFormer-base] model


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/935 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/224 [00:00<?, ?it/s]

...

Unfreezing last 2 of 24 transformer layers.


INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Output()

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Output()